In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import requests

In [4]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
soup=BeautifulSoup(source,'lxml')

In [145]:
#print(soup.prettify())

In [6]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [7]:
a = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [8]:
df = pd.DataFrame(a[0])

In [9]:
df.shape

(290, 3)

In [10]:
head_1 = df.iloc[0,:]
head_1

0         Postcode
1          Borough
2    Neighbourhood
Name: 0, dtype: object

In [11]:
df.columns=head_1

In [12]:
df.drop([0],inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Removing Borough with Not Assigned and Blank values

In [13]:
df = df[df.Borough!='Not assigned']
df = df[df.Borough!= 0]
df.reset_index(drop = True, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Assigning missing neighbourhood a Borough value

In [14]:
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1

In [15]:
df_table = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df_table.shape

(103, 3)

In [17]:
import geocoder

In [18]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [ ]:
#get_latlng('M1B')

As stated in assignment description the above code for getting latitude and longitude has stuck for me. So i am using the given Geospatial Cordinates file to get required data.

In [20]:
df_geo=pd.read_csv('Geospatial_Coordinates.csv')

In [21]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df_geo.drop(['Postal Code'],axis=1,inplace=True)
df_geo.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [23]:
df=df_table.join(df_geo)

In [24]:
df.shape
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
